## ライブラリのインストール

In [1]:
import tarfile, json,os,urllib,requests,zipfile
from dotenv import load_dotenv, find_dotenv

In [2]:
#  ディレクトリから.envファイルを探す
dotenv_path = find_dotenv()

# 環境変数をロード
load_dotenv(dotenv_path)

# API_KEYに環境変数を代入
token = os.environ.get("AWR_token")

## APIコール用のURLを生成

In [3]:
API_project = '02_%E3%82%AB%E3%82%A4%E3%82%B7%E3%83%A3%E3%81%AE%E8%A9%95%E5%88%A4_Daily'
date = '2018-02-09'
action = 'get'

#url = 'https://api.awrcloud.com/v2/get.php?action=get&project={}&date={}&token={}'
url = 'https://api.awrcloud.com/get.php?action={}&project={}&date={}&token={}&compression=zip&version=2&file=0.tar.xz'
API_call = url.format(action,API_project,date,token)

## 格納先ディレクトリの作成とファイル名の指定

In [4]:
# ディレクトリの作成
directory = 'output/' + date + '/'
try:
    os.makedirs(directory)
except FileExistsError:
    pass

# ファイル名の指定
filename = date + '.zip'
# 階層も含めたディレクトリ名の指定
output =  directory + filename

## Zipファイルのダウンロード

In [5]:
r = requests.get(API_call, stream=True)
with open(output, 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)
            f.flush()

## Zipファイルの解凍

In [6]:
zfile = zipfile.ZipFile(output)
zfile.extractall(directory)

In [7]:
target_file = "%E3%82%A2%E3%82%A6%E3%83%88%E3%82%BD%E3%83%BC%E3%82%B7%E3%83%B3%E3%82%B0%E3%83%86%E3%82%AF%E3%83%8E%E3%83%AD%E3%82%B8%E3%83%BC+%E5%8F%A3%E3%82%B3%E3%83%9F_google%7CJP%7CShinjuku%2C+Tokyo%2C+Japan%7Cja%7Corganic%7C1%7C1%7C0.json"

In [8]:
with open(directory + target_file) as f:
   data = json.load(f)

In [9]:
import pandas as pd
rankdata = pd.DataFrame.from_dict(data['rankdata'])

In [10]:
rankdata['searchengine'] = data['searchengine']
rankdata['keyword'] = data['keyword']